In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers

import pandas as pd
from pandas import read_csv

In [ ]:
# Convert from 64 to 32 bit floats
X_test = np.load("/home/alon/school/c247a/datasets/project_data/project/X_test.npy").astype(np.float32)
y_test = np.load("/home/alon/school/c247a/datasets/project_data/project/y_test.npy").astype(np.float32)
person_train_valid = np.load("/home/alon/school/c247a/datasets/project_data/project/person_train_valid.npy").astype(np.float32)
X_train_valid = np.load("/home/alon/school/c247a/datasets/project_data/project/X_train_valid.npy").astype(np.float32)
y_train_valid = np.load("/home/alon/school/c247a/datasets/project_data/project/y_train_valid.npy").astype(np.float32)
person_test = np.load("/home/alon/school/c247a/datasets/project_data/project/person_test.npy").astype(np.float32)

# adjust labels 
y_train_valid -= 769
y_test -= 769

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
print('y_train_valid', y_train_valid[:10])

In [ ]:
#convert y_train_valid to one-hot
y_train_valid = y_train_valid.astype(int)
y_train_valid_one_hot = np.zeros((y_train_valid.size, y_train_valid.max()+1))
y_train_valid_one_hot[np.arange(y_train_valid.size),y_train_valid] = 1
print(y_train_valid_one_hot.shape)

In [ ]:
n_timesteps = X_train_valid.shape[2]
n_features = X_train_valid.shape[1]
n_outputs = y_train_valid_one_hot.shape[1]

In [ ]:
plt.style.use(['seaborn-muted', 'seaborn-talk'])
plt.rcParams['figure.figsize'] = (16,9)

num_channels = 3
num_trials = 3
fig, ax = plt.subplots(num_channels, num_trials, sharex='col', sharey='row')
for i in range(num_trials):
  for j in range(num_channels):
    ax[j,i].plot(X_train_valid[i*700,j*7,:].T)
    if j == len(ax) - 1:
      ax[j,i].set_xlabel(f'Trial {i*700}, '
                         f'Subject {int(person_train_valid[i*700,0])}, '
                         f'Label {y_train_valid[i*700]}')
    if i == 0:
      ax[j,i].set_ylabel(f'Electrode {j*7}')
plt.tight_layout()

In [ ]:
model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(n_features,n_timesteps)))
model.add(layers.Dropout(0.9))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model = models.Sequential() #initialize sequential model
model.add(layers.LSTM(1024, input_shape=(n_features,n_timesteps), return_sequences=True)) #LSTM layer with 32 neurons
model.add(layers.LSTM(256, return_sequences=True)) #LSTM layer with 32 neurons
model.add(layers.LSTM(256)) #LSTM layer with 32 neurons
model.add(layers.Dropout(0.9))
model.add(layers.Dense(500,activation='relu')) #Dense layer with 16 neurons
model.add(layers.Dense(100,activation='relu')) #Dense layer with 8 neurons
model.add(layers.Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
def ksquare(x):
    return tf.pow(x, 2)

def klog(x):
    return tf.math.log(x)

In [ ]:
class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM, self).__init__()

        self.lstm = nn.LSTM(22, 64, 3, batch_first=True, dropout=DROPOUT)

        self.fc = nn.Sequential(
            nn.Linear(64, 54),
            nn.BatchNorm1d(num_features=54, eps=1e-05, momentum=0.2, affine=True),
            nn.ReLU(inplace = True),
            nn.Dropout(p=DROPOUT),
            nn.Linear(54, 44),
            nn.BatchNorm1d(num_features=44, eps=1e-05, momentum=0.2, affine=True),
            nn.ReLU(inplace = True),
            nn.Linear(44, 4)
        )
    
    def forward(self, x, h=None):

        # LSTM
        N, C, H, W = x.size()
        x = x.view(N, H, W).permute(0, 2, 1)
        out, _ = self.lstm(x)

        # Fully Connected Layer
        out = self.fc(out[:, -1, :])

        return out


In [ ]:
model = models.Sequential() #initialize sequential model
#model.add(layers.Reshape(input_shape=(n_features,n_timesteps), target_shape=(22, 1000, 1)))
model.add(layers.Conv1D(input_shape=(n_features,n_timesteps), filters=40, kernel_size=, data_format='channels_last',
                   activation='elu'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
loss_hist = model.fit(X_train_valid, y_train_valid_one_hot, 
                      validation_split=0.2, epochs=100,
                      verbose=True)

In [ ]:
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
# list all data in history
print(loss_hist.history.keys())
# summarize history for accuracy
plt.plot(loss_hist.history['accuracy'])
plt.plot(loss_hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(loss_hist.history['loss'])
plt.plot(loss_hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()